# TPM034A Machine Learning for socio-technical systems
## `Lab session 03:  Working with image embeddings`

**Delft University of Technology**<br>
**Q2 2023**<br>
**Instructor:** Sander van Cranenburgh <br>
**TAs:**  Francisco Garrido Valenzuela & Lucas Spierenburg <br>

### `Instructions`

**Lab sessions aim to:**<br>
* Show and reinforce how models and ideas presented in class are used in practice.<br>
* Help you gather hands-on machine learning skills.<br>

**Lab sessions are:**<br>
* Learning environments where you work with Jupyter notebooks and where you can get support from TAs and fellow students.<br>
* Not graded and do not have to be submitted.

### `Use of AI tools`
AI tools, such as ChatGPT and Co-pilot, are great tools to assist with programming. Moreover, in your later careers you will work in a world where such tools are widely available. As such, we **encourage** you to use AI tools **effectively** (both in the lab sessions and assignments). However, be careful not to overestimate the capacity of AI tools! AI tools cannot replace you: you still have to conceptualise the problem, dissect it and structure it, to conduct proper analysis and modelling. We recommend being especially **reticent** with using AI tools for the more conceptual and reflective oriented questions.

### `Workspace set-up`

**Option 1: Local environment**<br>
Uncomment the following cell if you are running this notebook on your local environment. This will install all dependencies on your Python version.

In [ ]:
#!pip install -r requirements.txt

**Option 2: Google Colab**<br>
Uncomment the following cells code lines if you are running this notebook on Colab

In [1]:
!git clone https://github.com/TPM034A/Q2_2023
!pip install -r Q2_2023/requirements_colab.txt
!mv "/content/Q2_2023/Lab_sessions/lab_session_03/data" /content/data
!mv "/content/Q2_2023/Lab_sessions/lab_session_03/assets" /content/assets

Cloning into 'Q2_2023'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 218 (delta 28), reused 53 (delta 27), pack-reused 159
Receiving objects: 100% (218/218), 94.70 MiB | 12.59 MiB/s, done.
Resolving deltas: 100% (74/74), done.
Updating files: 100% (59/59), done.


### `Application: Predicting perceived visual neighbourhood attractiveness` <br>

In this lab session we will use various ML models, namely Linear regressions (LR), Random forests (RF), Multi-layer perceptrons (MLP), and Ensembles (E), to predict the **perceived visual attractiveness of neighbourhoods**. Understanding the visual attractiveness of neighbourhoods is important for various reasons. For instance, municipalities need to know which neighbourhood need attention because they are visually unattractive. Moreover, visual attractiveness of neighbourhood is important to understand and predict residential location choices, house prices, and tourist destination choices.  

`In this lab session we aim to develop a computationally efficient ML model that is capable of mapping urban images (i.e. a Google Street view image) to visual attractiveness levels`.

**Where do the true labels come from?**<br>

Since we use supervised learning, we need to have data containing the true labels. But where do the true labels for visual attractiveness come from? The true labels come from a computer vision model that is trained on data from a so-called discrete choice experiment. In this experiment, people were placed in the hypothetical situation that they had to move to a different neighbourhood and were given two options. The visual attractiveness is learned from their choices.

See the figure below for an example of one such choice task. More information about the survey and the model can be found in [this paper](https://arxiv.org/abs/2308.08276).

<div>
<center><img src="assets/survey_snapshot.png" width="500"/></center>
</div>

**Why do we need a another model to (re)do the mapping?**<br>

The mapping from image to visual attractiveness indeed already comes from a model. Therefore, one may wonder why do we need another model? Applying the computer vision model is computationally expensive. Moreover, it requires a GPU to process images in large quantities. Therefore, having a good, and **computationally efficient**, proxy model is useful. For instance, to build a map like the one below for Delft requires processing over 400k images.

<div>
<center><img src="assets/visual_attractiveness_Delft.png" width="500"/></center>
</div>

**Data for this lab**<br>

In this lab session you will work with four datasets:

1. `data/image_tabular/image_metadata.csv`: A table csv file with image metadata (e.g., year, month or location) of Rotterdam images.
2. `data/image_tabular/image_embeddings.csv`: A table csv file with image embeddings of images from Rotterdam. (We will explain what image embeddings are later).
3. `data/geo/hexagons.gpkg`: A geospatial dataset of Rotterdam.
4. `data/images`: A folder with image files from Rotterdam (read below for more details).

The first three data files are already in the `data` folder associated with this lab. The `images` folder still needs to be downloaded. The full image data set is fairly large, 14 GB, and contains 101,444 images from Rotterdam. You can download the full dataset if you want to work and explore all the images by your own, but this is not required for completing this lab session. To conduct this lab session, we have created a sub-set of the data set containing 1,000 images which is only 140 MB. This allow you to conduct the visualisations of this lab.

The following cell will download the sub-set of images and place them in the `data` folder automatically. It can takes up to one minute to download the images. If you want to download the full dataset, just modify the the variable `download_full_dataset` to `True`.

In [2]:
## IMPORTANT: You have to be on the TUDelft network (eduroam) or under eduVPN to run this script
download_full_dataset = False

from assets import image_downloader as imd
imd.download_images(download_full_dataset)

Download complete!
Unzipping images...
Unzip complete!
Removing zip file...
Done!


**Learning objectives**. After completing the following exercises you will be able to: <br>
1. Train multiple ML models, including `Linear regressions`, `Random Forests` and `Multi-Layer Perceptrons`
2. Identify the most important features and their impact on the target feature<br>
3. Work with embeddings of images
4. Explore the pros and cons of each model, and how models can complement each other to answer specific research questions <br>

#### `Organisation`
This lab session comprises **`3`** parts:

1. Loading and exploring the data<br>
    1.1. Reading the medatada and geospatial file<br>
    1.2. Exploring the image metadata<br>
    1.3. Exploring the geospatial data<br>
    1.4. Visual inspection of images<br>

2. Image embeddings<br>
    2.1. Embedding model<br>
    2.3. Exploring the embeddings<br>

3. Training multiple models: Regression model, Random forest and MLP for predicting attractiveness<br>
    3.1. Preparing the dataset<br>
    3.2. Linear multiple regression model<br>
    3.4. Random Forest<br>
    3.5. Multilayer perceptrons<br>
    3.6. Comparing and reflecting on the model performances and their outcomes<br>

In [ ]:
# Basic libraries
import numpy as np
import pandas as pd
import geopandas as gpd

# ML tools
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer,log_loss

# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.neural_network import MLPRegressor

# Visualization libraries
import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.tree import plot_tree
from branca.element import Figure

# Other libraries
from pathlib import Path
from shapely.geometry import Point
from PIL import Image
import pickle

#### **1. Loading and exploring the data**
##### 1.1. Reading the medatada and geospatial data

Before creating models, we must understand our datasets. So, first open the image metadata dataset (*data/image_tabular/image_metadata.csv*) which contains general info (metadata) about the images, and the geospatial dataset (*data/geo/hexagons.gpkg*) which contains the spatial zones we will work with.

In [ ]:
# Data folder path
data = Path('data')

# Reading image df
img_metadata = pd.read_csv(data/'image_tabular'/'image_metadata.csv')

# Reading hexagons gdf
hexagons = gpd.read_file(data/'geo'/'hexagons.gpkg')

##### 1.2. Exploring the image metadata

The `img_metadata` DataFrame contains the metadata of thousands of Street View Images (SVI) from Rotterdam. With `head()` we can explore its different columns available.

In [ ]:
img_metadata.head(5)

The dataset contains:

- **img_id**: A unique identifier of an individual image
- **img_path**: Image filename in the image folder
- **year**: Year when the picture was taken
- **month**: Month when the picture was taken
- **lat**: geospatial latitude of the image
- **lng**: geospatial longitude of the image
- **hex_id**: geospatial hexagon id where the image was taken (see more details below)
- **attractiveness**: Numerical value which represent the perceived attractiveness of the image
- **in_folder**: Binary column indicating if the image is in the sub-set of images or not

We look at general statistics of `img_metadata` using `describe()`:

In [ ]:
img_metadata.describe()

In [ ]:
# Histogram of the attractiveness at image level
fig, ax = plt.subplots(figsize=(6, 3))
ax.hist(img_metadata['attractiveness'], bins=100)
ax.set_xlabel('Visual attractiveness')
ax.set_ylabel('Frequency')
ax.set_title('Histogram of the attractiveness at image level')
plt.show()

Here we can see:
- that the distribution of the attractiveness values is skewed a bit to the right. This means that there are more images with a more positive attractiveness value than negative.


Now let's briefly see how many images we have per geospatial hexagon. For this we will use the `hex_id` column. This column contains a unique identifier for each hexagon.

### <span style="color:skyblue">Exercise 1: Explore the statistics and geospatial distribution of the images</span>
`A` First, verify how many images are available per hexagon. Visualize by table or plot. <br>
`B` Create histograms of images by year and month. Comment what you see. <br>
`C` Visualize on a map the distribution of year and month (To convert the DataFrame into a GeoDataFrame use the method provided below). Create one map for each year (2008-2022), and for each month (Jan-Dec). Interpret your results.<br>
`D` Do you think the month and year of the images could impact on the (predicition of) perceived visual attractiveness?<br>

In [ ]:
def dataframe_to_geodataframe_nl(original_dataframe, latitud_column_name, longitude_column_name):
    '''
    This Function converts a dataframe into a geodataframe using the latitud and longitude columns.
    The output will be ready to use the plot function from geopandas.
    '''

    ## Creating the Point geometry using lat/lng columns
    original_dataframe['geometry'] = original_dataframe.apply(lambda x: Point(x[longitude_column_name], x[latitud_column_name]), axis=1)

    ## Creating the geodataframe (we used crs 4326 because it is the code for the latitud and longitude)
    geodataframe = gpd.GeoDataFrame(original_dataframe, geometry='geometry', crs=4326)

    ## Changing the crs to the same as the hexagons This is 28992, the projection used in the Netherlands.
    geodataframe = geodataframe.to_crs(28992)

    return geodataframe

In [ ]:
# ADD HERE YOUR ANSWER TO EXERCISE 1

# A.


In [ ]:
# B.

In [ ]:
# C.

##### 1.3. Exploring the geospatial data

The second dataset *hexagons* corresponds to a geospatial dataset of hexagons.

In [ ]:
hexagons.head(5)

The dataset contains only two columns. The first one, **hex_id**, it is just a unique identifier for each hexagon. The second one, **geometry**, corresponds to the coordinates of the shape and location of each hexagon. The following code, allow us to visualize each hexagon on the map using the method `explore()` from GeoPandas

In [ ]:
fig = Figure(width=400, height=300)
fig.add_child(hexagons.explore(marker_kwds={'radius': 10}, zoom_start=15))
display(fig)

As you can see in the map, each hexagon covers a piece of land in the city. During this lab, we will treat a hexagon as a neighbourhood. As we described before, each image from the *images* DataFrame is associated with an hexagon id (*hex_id column*), meaning that each image belongs to a neighbourhood. Now it is time to explore the (geo)relation of these two datasets.

### <span style="color:skyblue">Exercise 2: Do you recongnize some geospatial patterns of the attractiveness?</span>
`A` Compute the average attractiveness *per hexagon*<br>
`B` Merge both DataFrames on the column *hex_id*<br>
`C` Visualise the average attractiveness of each neighbourhood, using the `explore()` method of geopandas<br>
`D` Do you see spatial patterns in the map? Hint: check for areas with parks, highways, and other places that you know. Does the map make intuitive sense to you?<br>

In [ ]:
# ADD HERE YOUR ANSWER TO EXERCISE 1

# A.


##### 1.4. Visual inspection of images and attractiveness

The full image dataset contains 101,444 images. To easily visualise street images in your computer we have created a representative sample with 1,000 images located on the image folder inside data folder. Let's see an example how to visualize an image in an IPython notebook.

In [ ]:
image_path = lambda file: data/'images'/file
visual_images = img_metadata.loc[img_metadata['in_folder'] == 1].copy()

Run the following cell several times to see different images.

In [ ]:
# Create a figure
fig, ax = plt.subplots(1,2,figsize=(12,3.5))

# Draw a random image
rnd_img = visual_images.sample(1)[['img_path','attractiveness']]

# Visualising the random image
img = Image.open(image_path(rnd_img.iloc[0]['img_path'])).convert('RGB')
ax[0].imshow(img)

# Remove the axis
# ax[0].axis('off')
ax[0].set_xlabel('Random image')
ax[0].set_xticks([])
ax[0].set_yticks([])

# Add the histogram of the attractiveness of all 101k images
sns.histplot(img_metadata['attractiveness'],bins=50, stat='density', ax=ax[1])
ax[1].set_xlabel('Visual attractiveness')

# Show where its attractiveness is located in the distribution
img_attractiveness = rnd_img.iloc[0]['attractiveness']
bin_edges = ax[1].patches
for patch in bin_edges:
    if patch.get_x() <= img_attractiveness <= patch.get_x() + patch.get_width():
        patch.set_facecolor('red')
plt.show()

**Reflection**<br>
Across the board the location in the distribution makes sense. But, some unattractive streetviews are predicted as being fairly attractive (and vice versa).

Now we go one step further, and visualise images based on its levels of attractiveness. Modify the number of percentiles (`n_percentiles`) and the number of images per percentile (`images_per_row`) to explore different groups of images.

Images from lower percentile are less attractive than images from higher percentiles based on its attractiveness value. Take a look to this images and see if makes sense to you.

In [ ]:
# Set the number of percentiles and images per row
n_percentiles = 6
images_per_row = 5

# Create a new column with the quantile
visual_images['quantile'], quantile_boundaries = pd.qcut(visual_images['attractiveness'], n_percentiles, labels=False, retbins=True)

# Create a figure with a grid
fig = plt.figure(figsize=(20., 100.))
grid = ImageGrid(fig, 111, nrows_ncols=(n_percentiles,images_per_row+1),axes_pad=0.05)

#
for i in range(n_percentiles):
    # Draw 'images_per_row' random images from the quantile i
    auxs = visual_images[visual_images['quantile']==i].sample(images_per_row)[['img_path', 'attractiveness']].reset_index(drop=True)

    # Plot the images in the grid
    for j, row in auxs.iterrows():
        img = Image.open(image_path(row['img_path'])).convert('RGB')
        grid[(i*(images_per_row+1))+j+1].imshow(img)
        grid[(i*(images_per_row+1))+j+1].axis('off')

    # Add the quantile label and the quantile boundaries
    grid[i*(images_per_row+1)].text(-300, 300, f'Quantile {i+1}\n ({quantile_boundaries[i]:0.2f}, {quantile_boundaries[i+1]:0.2f})', fontsize=15, ha='center', va='center')

    # Do not show axis
    grid[i*(images_per_row+1)].axis('off')

# Show the figure
plt.show()

We can inspect the tails of the distribution. That is, we can inspect the **most attractive** images and the **least attractive** images.

Here we see the 10 least attractive images.

In [ ]:
# Set the number of images per row
images_per_row = 5

# Sort the images by attractiveness and get the first and last images
mins = visual_images.sort_values(by='attractiveness', ascending=True).head(images_per_row*2).reset_index(drop=True)

# Create a figure with a grid
fig = plt.figure(figsize=(20., 100.))
grid = ImageGrid(fig, 111, nrows_ncols=(2,images_per_row),axes_pad=0.05)

# Plot the images in the grid
for i, row in mins.iterrows():
    img = Image.open(image_path(row['img_path'])).convert('RGB')
    grid[i].imshow(img)

    # Do not show axis
    grid[i].axis('off')

# Show the figure
plt.show()

In [ ]:
# Set the number of images per row
images_per_row = 5

# Sort the images by attractiveness and get the first and last images
maxs = visual_images.sort_values(by='attractiveness', ascending=True).tail(images_per_row*2).reset_index(drop=True)

# Create a figure with a grid
fig = plt.figure(figsize=(20., 100.))
grid = ImageGrid(fig, 111, nrows_ncols=(2,images_per_row),axes_pad=0.05)

# Plot the images in the grid
for i, row in maxs.iterrows():
    img = Image.open(image_path(row['img_path'])).convert('RGB')
    grid[i].imshow(img)

    # Do not show axis
    grid[i].axis('off')

# Show the figure
plt.show()

**Question**<br>
Do you agree with the model's assessment of the least and most visually attractive images? Why?

#### **2. Image embeddings**
##### 2.1. Embedding model

Until now we have only visualised the images. But we would like to work with them. For that we need to transform the images into lower dimensional mathematical representations that allows for algorithmic processing. This mathematical representation is called an embedding. For instance, an image has 720 x 320 x 3 (heigh x width x color channels) pixels, while an embedding of that image typically contain only between 20 and 2,000 floating points. The figure shows conceptually shows how a classic Convolution Neural Network (CNN) produces an image embedding - which is also called **feature map**.



The image embedding that we will work with is generated by the computer model that produces the visual attractiveness scores. More generally, embedding producesed by trained models are widely used for other related tasks, such as clustering, similarity search, etc.

<div>
<img src="assets/image_embbeding.png" width="500"/>
</div>

To obtain the embeddings, we need to apply the trained computer vision model to the images we are working with in this lab session. However, because this is technically beyong the scope of this course, we have already proccesed the images in advance and saved the embeddings in a csv file for you. In the following cell we load the embeddings.

In [ ]:
# Load the embeddings
embeddings = pd.read_csv(data/'image_tabular'/'image_embeddings.csv')

##### 2.1. Exploring the embeddings

In [ ]:
embeddings.head()

As we can see, the embeddings have 25 elements. It means, each image is represented by 25 latent features. We don't know what these features semantically mean. But we can use them to work with the images.

In [ ]:
embeddings.describe()

### <span style="color:skyblue">Exercise 3: Are year or month variables encoded by the images?</span>
`A` Append the year, month and attractiveness to the image embeddings columns <br>
`B` Create a correlation matrix between all the variables<br>
`C` Do you think the month, year or the attractiveness are are contained in some features of the embedding?<br>

In [ ]:
## Answer to exercise 3

#### **3. Training multiple models: Regression model, Random forest and Multilayer perceptron (MLP) for predicting attractiveness**<br>
Often, a researcher does not know beforehand which sort of model will do well, and which will not. Therefore, ML researchers often apply multiple models to their task and pull together their outcomes.<br>

Next, we are going to apply 3 models:
1. Regression model (benchmark model)<br>
2. Random Forest (RF)<br>
3. Multilayer Perceptron (MLP)<br>

As we will see, each of these models provide complementary insights. (maybe delete)

##### 3.1. Preparing the dataset

In [ ]:
# Merging the embeddings with the original dataset
image_embeddings = pd.merge(img_metadata, embeddings, on='img_id')
image_embeddings.head(5)

In [ ]:
# Generating the embedding columns
df = image_embeddings.copy()
emb_cols = embeddings.drop(columns=['img_id']).columns
features = emb_cols.to_list()
predict = 'attractiveness'

# Setting all dataset between 0 and 1
scaler = StandardScaler()
df[features + [predict]] = scaler.fit_transform(df[features + [predict]])

# Splitting the dataset
X_train, X_test, Y_train, Y_test = train_test_split(df[features], df[predict], test_size=0.2, random_state=44)

Because we are going to compare different models, we create a custom evaluation function that allows us to swiftly report the following stats for the train and test data:
* mean square error
* mean absolute error
* R2

In [ ]:
def eval_regression_perf(model, X_train, X_test, Y_train, Y_test):

    # Make prediction with the trained model
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)

    # Create a function that computes the MSE, MAE, and R2
    def perfs(Y,Y_pred):
        mse = mean_squared_error(Y,Y_pred)
        mae = mean_absolute_error(Y,Y_pred)
        R2 = r2_score(Y,Y_pred)
        return mse,mae,R2

    # Apply the perfs function to the train and test data sets
    mse_train, mae_train, r2_train = perfs(Y_train,Y_pred_train)
    mse_test,  mae_test , r2_test  = perfs(Y_test,Y_pred_test)

    # Print results
    print('Performance')
    print(f'Mean Squared  Error Train | Test: \t{mse_train:>7.4f}\t|  {mse_test:>7.4f}')
    print(f'Mean Absolute Error Train | Test: \t{mae_train:>7.4f}\t|  {mae_test:>7.4f}')
    print(f'R2                  Train | Test: \t{ r2_train:>7.4f}\t|  {r2_test:>7.4f}\n')

##### 3.2. Linear multiple regression model
This model usually serves as a benchmark when creating ML models. Therefore, we start with this model.

In [ ]:
# Create and train a multiple linear regression model
regr = LinearRegression(fit_intercept = True)

# Fit the model on the training data
regr.fit(X_train,Y_train)

# Evaluate the performance of the trained model
eval_regression_perf(regr, X_train, X_test, Y_train, Y_test)

# We take an ML approach (i.e. we use a train and tests set). But, we can still look at the regression coefficients to see which features are more important than others.
regr_results_int = pd.DataFrame(index = ['Intercept'], data = regr.intercept_, columns = ['Coefficient'])
regr_results_wei = pd.DataFrame(index = df[features].columns, data = regr.coef_, columns = ['Coefficient'])
regr_results = pd.concat([regr_results_int,regr_results_wei],axis=0)

# Show the results
regr_results.sort_values(by='Coefficient', ascending=False).plot(kind='bar', figsize=(10, 4))

Based on these results, we can make several inferences:
1. The model fit relatively well (R2 of ~0.65)
2. The model does not seem to overfit the data: it attains the same performance on the test as on the train data sets
3. Based on the plot, we can infer that most feature are likely to contribute to the prediction performance, since they are nonzero.
4. Based on the plot, we can infer that, according to this model, Fhat004, Fhat002, and Fhat001 are the most important features.

##### 3.3. Random Forest (RF)
Now, we are going to try a RF on these data. RFs are known to attain a high model performance on a wide range of regression and classification tasks.
<br>
RFs need hyperparameter tuning. Instead of manually searching for the optimal hyperparameters for the RF, which is laborious, let's use sk-learn's GridSearchCV functionality to automate the hyperparameter search. In practice, typically, we want to search over the 4 most important hyperparameters: `max_depth`, `max_leaf_nodes`, `min_samples_leaf`, `max_features`. However, to avoid too long compuatational times during this lab session, here we restrict the tuning to the first two hyperparameters and try only two levels.  

In [ ]:
# Create RF object for hyperparameter tuning
rf_gs = RandomForestRegressor(n_estimators=50,criterion= "squared_error",random_state=5)

# Define the hyperparameter search space
hyperparameter_space = {
    'max_depth': [5, 20],
    'max_leaf_nodes': [500,2500],
    # 'min_samples_leaf': [15],     # Not tuned here too avoid long computation time
    # 'max_features':[0.7]          # Not tuned here too avoid long computation time
    }

# Create scoring function
scorer = make_scorer(mean_squared_error, greater_is_better = False)

# Create the grid_search object, with using the MLP classifier
folds = 5 # Number of cross validation splits
rf_gridsearch = GridSearchCV(rf_gs, hyperparameter_space, n_jobs=-1, cv=folds, scoring=scorer, return_train_score=True)

In [ ]:
# if the model RF_gridsearch.pickle does not exist, conduct the gridsearch, else load the grid_search object
if not Path('RF_gridsearch.pickle').exists():

    # Fit the grid search object to the training data
    # Note that this is computationally expensive!
    # It may take up to 5 minutes, since multiple models need to be trained multiple times
    rf_gridsearch.fit(X_train, Y_train)

    # Save the trained grid_search object
    with open('RF_gridsearch.pickle', 'wb') as f:
        pickle.dump(rf_gridsearch, f)
else:
    # Load the trained grid_search object
    with open('RF_gridsearch.pickle', 'rb') as f:
        rf_gridsearch = pickle.load(f)

# Print the best parameters
print('Best parameters:')
print(f'Tuned hyperparameters:\n{rf_gridsearch.best_params_}')
print(f'Mean Squared Error:\t{(-rf_gridsearch.best_score_):0.3f}')

In [ ]:
# Create a dataframe from the results
df_results = pd.DataFrame(rf_gridsearch.cv_results_)

# Extract relevant columns
df_results = df_results[['params', 'mean_test_score', 'mean_train_score']]

# Display the top performing models in a plot
top_models = df_results.sort_values(by='mean_test_score', ascending=False)

# Create a figure
fig, ax = plt.subplots(figsize=(10, 3))

# Convert the dict row names to a list of strings, and replace the commas with newlines
param_list = [str(dict_) for dict_ in top_models['params']]
param_list = [param.replace(',', '\n') for param in param_list]
param_list = [param.replace('{', '')   for param in param_list]
param_list = [param.replace('}', '')   for param in param_list]

# Create a barplot
sns.barplot(x= param_list, y=top_models['mean_test_score'], ax=ax)

# Set the x-axis labels to the parameter list
ax.set_xticklabels(param_list, rotation=90)

# Show the plot
plt.show()

In [ ]:
# Set the hyperparameters of the Random Forest to the best values found, e.g.
max_depth = rf_gridsearch.best_params_['max_depth']
max_leaf_nodes = rf_gridsearch.best_params_['max_leaf_nodes']
max_features = 0.7
min_samples_leaf = 15

# Create the Random Forest object with the best hyperparameters
rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth, max_features=max_features, max_leaf_nodes=max_leaf_nodes, min_samples_leaf=min_samples_leaf, random_state=0,n_jobs=-1)

# Train the Randon Forest
rf.fit(X_train,Y_train)

# Evaluate the performance of the hyperparameter optimised RF model
eval_regression_perf(rf,X_train,X_test, Y_train, Y_test)

Let's see which features are found to be most important to our Random Forest.

In [ ]:
# Plot the feature importances
fig, ax = plt.subplots(figsize=(8,10))
sns.barplot(y=features,x=rf.feature_importances_, ax = ax).set(title='Feature importance to predict Attractiveness')
ax.set_xlabel('Feature importance')

# Print most important features
sorted_indices = np.argsort(rf.feature_importances_)[::-1]
most_imp_features_rf = [features[i] for i in sorted_indices]
print(f'Top 5 most important features:\n {most_imp_features_rf[:6]}')

# Show plot
plt.show()

However, as explained before, the embeddings are not semantically interpretable.

Nonetheless, we can draw some inference from it:
1. In line wit the linear multiple regression, we see that Fhat004 is the most important feature for the prediction. In contrast with the linear regression, Fhat001 is less important to the RF.
1. A few features are strong predictors, while most others contribute only modestly to the prediction.

#### 3.4 Multilayer perceptions (MLP)

### <span style="color:skyblue">Exercise 4: Apply GridSearch now using MLP to predict the attractiveness</span>
More specifically, tune the size of the hidden layers, the regularisation, and the learning rate. Use for each of these hyperparameters, no more than two levels to avoid too long compuational times.

In [ ]:
# ANSWER TO EXERCISE 4

#### **3.5 Comparing and reflecting on the model performances and their outcomes**

Often it helps the researcher to pull together the outcomes of multiple models. The idea of doing this is closely related to wisdom of the crowd principle. If multiple different model point towards the same conclusion, then the researcher has compounding evidence. In contrast, if different models point towards different conclusions, then the researcher knows the conclusions are weak(er) and sensitive to the choice of model.<br>
Let's pull together the predictions of the 3 ML models, and show them next to one another.

In [ ]:
# Scatter true shares (x-axis) against the predicted shares by the different ML models (y-axis)
fig, ax = plt.subplots(1,3,figsize=(18,4), sharex=True, sharey=True)
ax[0].scatter(x = Y_test, y = regr.predict(X_test),  marker = '.', s = 25, alpha = 0.2, c = '#ff7f0e', label = f'Regression | R2 = {regr.score(X_test,Y_test):0.3f}')
ax[1].scatter(x = Y_test, y = rf.predict(X_test),    marker = '.', s = 25, alpha = 0.2, c = '#2ca02c', label = f'Random Forest| R2 = {rf.score(X_test,Y_test):0.3f}')
ax[2].scatter(x = Y_test, y = mlp_gs.predict(X_test),marker = '.', s = 25, alpha = 0.2, c = '#d62728', label = f'MLP | R2 = {mlp_gs.score(X_test,Y_test):0.3f}')

# Add labels, legend, and title to each plot
for n in range(0,3,1):
    ax[n].set_xlabel('True Attractiveness')
    ax[n].set_ylabel('Predicted attractiveness')
    ax[n].set_xlim(-4,4)
    ax[n].set_ylim(-4,4)
    ax[n].legend()
    ax[n].set_title('True versus predicted attractiveness (test data)')

plt.show()

### <span style="color:skyblue">Exercise 5:  Comparison of predictions</span>
`A` What is the most striking difference between the predictions across the three ML models? Can you explain this?<br>
`B` How would the (ideal) scatter plot look like? I.e. when the model would perfectly predict our target?


In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)


Another way the compare the predictions of the three models is by creating a **kernel density plot**. A kernel density plot visualises the distribution, i.e. the probability density, of a variable, in a smooth way, see e.g. [wikipedia.org/wiki/Kernel_density_estimation](https://en.wikipedia.org/wiki/Kernel_density_estimation).

In [ ]:
# Plot kernel densities for the test data
fig, ax = plt.subplots(figsize=(10, 7))

# Plot kernel density of the true distribution, i.e of Y_test
sns.kdeplot(data = Y_test, ax=ax, legend=True,fill=True,alpha=.3, clip =(-4,4), label = 'True')

# Plot kernel densities of the predicted distributions, based on the three models
sns.kdeplot(data = regr.predict(X_test),ax=ax, legend=True,fill=True,alpha=.3, clip =(-4,4), label = f'Regression | R2 = {regr.score(X_test,Y_test):0.3f}')
sns.kdeplot(data = rf.predict(X_test),  ax=ax, legend=True,fill=True,alpha=.3, clip =(-4,4), label = f'Random Forest | R2 = {rf.score(X_test,Y_test):0.3f}')
sns.kdeplot(data = mlp_gs.predict(X_test),  ax=ax, legend=True,fill=True,alpha=.3, clip =(-4,4), label = f'MLP | R2 = {mlp_gs.score(X_test,Y_test):0.3f}')

# Add legend and title to the plot
#ax.legend(['True','Regression model', 'Decision tree','Random Forest'])
ax.legend()
plt.title('True vs predicted Attractiveness')
ax.set_xlim(-4,4)
plt.show()

### <span style="color:skyblue">Exercise 6:   Conclusion and reflection on the results</span>
In this lab session we have analysed urban visual data, and made predictions for attractiveness using three ML models.<br>
`A` Based on the above analyses, which of these models can best be used to predict visual attractiveness of the urban space? Explain your answer.<br>
`B` In the kernel density plots, we see that the true variance of attractiveness (in blue) is considerably larger than the variance of attractiveness predicted by any of the ML models. What does this tell us about this regression problem?<br>
`C` How has applying multiple ML models helped you achieving your research goal?<br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)